# 🚀 Quick Start - Bild mit x402 Payment generieren

**So verwendest du dieses Notebook:**

1. **Zelle 2**: Imports laden ✅
2. **Zelle 3**: Network wählen (`USE_MAINNET = False` für Testnet) ✅
3. **Zelle 4**: Service-Verfügbarkeit prüfen (optional)
4. **Zelle 5**: 🎨 **HAUPT-ZELLE → Bild generieren mit automatischem Payment!**

---

## ⚡ Zelle 5 ist die wichtigste!

Diese Zelle führt den **kompletten x402 Payment Flow** aus:
- ✅ Sendet Request an Image Service
- ✅ x402 fängt `402 Payment Required` ab
- ✅ Signiert USDC Payment automatisch (EIP-3009)
- ✅ Wiederholt Request mit Payment Proof
- ✅ Service verifiziert via Facilitator
- ✅ Generiert Bild mit Black Forest Labs
- ✅ Mintet NFT an deine Wallet
- ✅ Gibt Bild-URL + Token ID zurück

**Du musst nur Zelle 5 ausführen** - der Rest passiert automatisch! 🎉

---

## 📚 Zusätzliche Zellen (optional)

Die Zellen darunter sind zur **Erklärung und zum Debuggen**:
- Details zum x402 Flow
- Manuelle 402 Response Inspektion
- Troubleshooting Guides

# x402 Token Payment Image Generation Demo

Dieses Notebook demonstriert den **x402 v2 Token Payment Flow** für die ImageGen API.

## Was ist x402 v2 Token Payment?

Die neue Implementation verwendet **USDC Token-Transfers** statt NFT Mints:
- Server antwortet mit `402 Payment Required` + x402 v2 Payment Header
- **🌐 Server bietet BEIDE Networks an:** Optimism Mainnet **und** Sepolia Testnet
- **Client wählt das Network** basierend auf `USE_MAINNET` Flag
- Client erstellt EIP-3009 signierte USDC-Transfer für gewähltes Network
- Client sendet Request mit Payment Proof an Service
- Service verifiziert Payment via Facilitator (gegen gewähltes Network!)
- Service generiert Bild + mintet NFT an Client
- Service settled Payment via Facilitator (async)

## Multi-Network Support (NEU!)

**Server-Side:**
Der Server bietet in der 402 Response **beide Networks gleichzeitig** an:
```javascript
accepts: [
  { network: "eip155:10", asset: "0x0b2C...Ff85" },      // Optimism Mainnet
  { network: "eip155:11155420", asset: "0x5fd8...30D7" } // Optimism Sepolia
]
```

**Client-Side:**
Der x402 Client wählt automatisch das passende Network basierend auf deiner Konfiguration:
- `USE_MAINNET = False` → Client signiert für Sepolia
- `USE_MAINNET = True` → Client signiert für Mainnet

**Verification:**
Der Server extrahiert das gewählte Network aus dem Client-Payment und verifiziert gegen die richtige Chain!

## Voraussetzungen

**Service (lokal testen):**
```bash
cd scw_js
NODE_ENV=test node ./genimg_x402_token.js
```
→ Service läuft auf `http://localhost:8082`
→ Verwendet Production Facilitator: `https://facilitator.fretchen.eu`

**Environment:**
- `TEST_WALLET_PRIVATE_KEY` in `scw_js/.env` (für Payment Signing)
- `NFT_WALLET_PRIVATE_KEY` in `scw_js/.env` (für NFT Minting auf Server)
- `FACILITATOR_URL=https://facilitator.fretchen.eu` in `.env` (default)
- Optimism RPC Zugriff
- BFL API Token (für Bildgenerierung)

**Netzwerk-Auswahl:**
- **Testnet:** `USE_MAINNET = False` → Optimism Sepolia (kein echtes Geld)
- **Mainnet:** `USE_MAINNET = True` → Optimism Mainnet (⚠️ echtes USDC!)

## Flow

1. **Request ohne Payment** → `402 Payment Required` mit x402 v2 header (beide Networks!)
2. **Client wählt Network** → x402 Client selektiert passendes Network aus `accepts` Array
3. **USDC Transfer signieren** (EIP-3009) mit x402 Python Client für gewähltes Network
4. **Request mit Payment** → Facilitator verifiziert gegen gewähltes Network, Service mintet NFT + generiert Bild
5. **Async Settlement** → Facilitator settled Payment on-chain

## Python x402 Package

**Verfügbar:** `pip install x402` (v1.0.1, by Coinbase)

**Perfekt für unseren Use Case:**
- `x402.clients.requests.x402HTTPAdapter`: Automatisches Network-Selection + Signing
- Unterstützt Optimism + Base (Mainnet & Testnet)
- Verwendet Facilitator für Verification & Settlement
- **Wählt automatisch das richtige Network** aus Server's `accepts` Array!

In [14]:
import requests
import json
from pprint import pprint
import os
from dotenv import load_dotenv
from eth_account import Account
from web3 import Web3

load_dotenv()
print(f"✅ Environment loaded")

✅ Environment loaded


In [15]:
PROMPT = "A small furry elephant pet looks out from a cat house"

# ⚠️ NETWORK SELECTION - Change this to switch between testnet and mainnet
USE_MAINNET = False  # Set to True for Optimism Mainnet with REAL MONEY

if USE_MAINNET:
    CHAIN_ID = 10  # Optimism Mainnet
    NETWORK_NAME = "Optimism Mainnet"
    print(f"\n🚨 WARNING: Using REAL MONEY on {NETWORK_NAME}!")
else:
    CHAIN_ID = 11155420  # Optimism Sepolia
    NETWORK_NAME = "Optimism Sepolia (Testnet)"
    print(f"\n🧪 Using testnet: {NETWORK_NAME}")

print(f"Chain ID: {CHAIN_ID}")

# Load private key from environment
private_key = os.getenv('TEST_WALLET_PRIVATE_KEY')
if not private_key:
    print("❌ TEST_WALLET_PRIVATE_KEY not found in .env")
    private_key = None
else:
    print(f"✅ Private Key loaded")
account = Account.from_key(private_key)
from_address = Web3.to_checksum_address(account.address)

# URLs for local service testing with production facilitator
FACILITATOR_URL = "https://facilitator.fretchen.eu"  # Production
SERVICE_URL = "http://localhost:8082"  # Local testing

print(f"\n🚀 x402 v2 Token Payment Flow Demo")
print(f"🌐 Network: {NETWORK_NAME}")
print(f"🏦 Facilitator: {FACILITATOR_URL}")
print(f"📍 Service: {SERVICE_URL}")
print(f"Payer Address: {from_address}")
print(f"✍️  Prompt: {PROMPT}\n")


🧪 Using testnet: Optimism Sepolia (Testnet)
Chain ID: 11155420
✅ Private Key loaded

🚀 x402 v2 Token Payment Flow Demo
🌐 Network: Optimism Sepolia (Testnet)
🏦 Facilitator: https://facilitator.fretchen.eu
📍 Service: http://localhost:8082
Payer Address: 0x553179556FC2A39e535D65b921e01fA995E79101
✍️  Prompt: A small furry elephant pet looks out from a cat house



In [ ]:
# 🔍 SCHRITT 1: Service Verfügbarkeit Check
print("🔍 Prüfe Service-Verfügbarkeit...\n")

# Check Facilitator (Production) - Use /supported endpoint
try:
    resp = requests.get(f"{FACILITATOR_URL}/supported", timeout=2)
    if resp.status_code == 200:
        supported = resp.json()
        print(f"✅ Facilitator: {FACILITATOR_URL} - Erreichbar")
        print(f"   Unterstützte Networks: {', '.join(supported.get('networks', []))}\n")
    else:
        print(f"⚠️  Facilitator: {FACILITATOR_URL} - Status {resp.status_code}\n")
except Exception as e:
    print(f"❌ Facilitator: {FACILITATOR_URL} - NICHT ERREICHBAR")
    print(f"   Error: {e}\n")

# Check Service (Local)
try:
    resp = requests.post(SERVICE_URL, json={"prompt": "test"}, timeout=2)
    print(f"✅ Service: {SERVICE_URL} - Erreichbar (Status {resp.status_code})\n")
    print("🎯 Alle Services bereit! Fahre mit Zelle 5 fort für die Bildgenerierung.\n")
except:
    print(f"❌ Service: {SERVICE_URL} - NICHT ERREICHBAR")
    print("   → Starte Service: cd scw_js && NODE_ENV=test node genimg_x402_token.js\n")

In [ ]:
# 🎨 SCHRITT 2: Bild generieren mit x402 Payment (HAUPT-ZELLE!)
print("\n" + "=" * 70)
print("🎨 BILDGENERIERUNG MIT X402 PAYMENT - KOMPLETTER FLOW")
print("=" * 70)

try:
        from x402.clients.requests import x402HTTPAdapter
        from x402.clients.base import x402Client
        
        print(f"\n📤 Erstelle x402 Session...")
        print(f"   Netzwerk: {NETWORK_NAME}")
        print(f"   Facilitator: {FACILITATOR_URL}")
        print("   → x402 Adapter fängt 402 ab, signiert Payment, wiederholt Request\n")
        
        # Create x402 client with private key and facilitator
        client = x402Client(private_key, FACILITATOR_URL)
        
        # Create requests session with x402 adapter
        # Adapter handles 402 responses automatically
        session = requests.Session()
        session.mount('http://', x402HTTPAdapter(client))
        session.mount('https://', x402HTTPAdapter(client))
        
        # Now use normal requests API with clean json= parameter!
        print("📤 Sende Request an Service...")
        print(f"   Prompt: \"{PROMPT}\"")
        print("\n⏳ Warte auf Response (kann 10-30 Sekunden dauern)...\n")
        
        response = session.post(
            SERVICE_URL, 
            json={"prompt": PROMPT},  # ✅ Sauberes JSON Handling!
            timeout=60
        )
        
        print("=" * 70)
        print(f"📬 STATUS: {response.status_code}")
        print("=" * 70)
        
        if response.status_code == 200:
            print("\n✅ ERFOLG! Bildgenerierung abgeschlossen!")
            result = response.json()
            print("\n" + "🎨 ERGEBNIS ".center(70, "="))
            pprint(result, width=70, indent=2)
            
            if 'image_url' in result:
                print(f"\n🖼️  Bild URL: {result['image_url']}")
            if 'metadata_url' in result:
                print(f"📋 Metadata: {result['metadata_url']}")
            if 'tokenId' in result:
                print(f"🎫 Token ID: {result['tokenId']}")
                
                # Network-specific explorer links
                if USE_MAINNET:
                    explorer_base = "https://optimistic.etherscan.io"
                else:
                    explorer_base = "https://sepolia-optimism.etherscan.io"
                
                contract_addr = "0x80f95d330417a4acEfEA415FE9eE28db7A0A1Cdb"
                print(f"\n🔗 View on Etherscan:")
                print(f"   {explorer_base}/token/{contract_addr}?a={result['tokenId']}")
                
        elif response.status_code == 402:
            print("❌ Payment Verification fehlgeschlagen")
            error = response.json()
            print("\n📄 Fehlermeldung:")
            pprint(error, width=70, indent=2)
            
            print("\n💡 Mögliche Ursachen:")
            print("   • Keine USDC Balance auf dem gewählten Netzwerk")
            print(f"   • Aktuelles Netzwerk: {NETWORK_NAME}")
            print("   • Facilitator nicht erreichbar")
            print("   • Netzwerk-Mismatch (Service vs. Client)")
            print("   • Insufficient allowance (sollte automatisch gesetzt werden)")
            
        else:
            print(f"❌ Unerwarteter Status: {response.status_code}")
            print("\n📄 Response:")
            print(response.text[:500])
            
except ImportError as e:
        print("❌ x402 Package nicht installiert!")
        print("   → pip install x402")
        print(f"   Error: {e}")
except requests.exceptions.Timeout:
        print("⏱️  Timeout - Bildgenerierung dauert normalerweise ~10-30 Sekunden")
        print("   Server arbeitet möglicherweise noch...")
        print("   → Prüfe Terminal wo Service läuft")
except requests.exceptions.ConnectionError:
        print("❌ Verbindungsfehler!")
        print("   Service erreichbar? → cd scw_js && NODE_ENV=test node genimg_x402_token.js")
        print("   Facilitator erreichbar? → Check https://facilitator.fretchen.eu")
except Exception as e:
        print(f"❌ Error: {e}")
        print("\n📄 Traceback:")
        import traceback
        traceback.print_exc()

print("\n" + "=" * 70)


SCHRITT 3: Request mit x402 Payment Handling

📤 Erstelle x402 Session...
   Netzwerk: Optimism Sepolia (Testnet)
   → x402 Adapter fängt 402 ab, signiert Payment, wiederholt Request

📤 Sende Request (x402 Session mit automatischem Payment)...
📬 Status Code: 500

❌ Unerwarteter Status: 500
Response:
{"error":"Cannot read properties of undefined (reading 'scheme')"}


## Schritt 3: Sende Request mit x402 Session

Statt `x402_requests()` verwenden wir **x402HTTPAdapter** mit einer normalen requests Session:

**Vorteile:**
- ✅ Sauberes `json=` Parameter Handling
- ✅ Normale requests API (Python-idiomatisch)
- ✅ Automatisches 402 Handling durch Adapter
- ✅ Stabiles EIP-3009 Signing (durch x402 Package)

**Wie es funktioniert:**
1. Erstelle `x402Client` mit Private Key + Facilitator URL
2. Mounte `x402HTTPAdapter` an requests Session
3. Verwende Session wie normale requests (mit `json=`!)
4. Adapter fängt 402 ab → signiert Payment → wiederholt Request

**Code:**
```python
from x402.clients.requests import x402HTTPAdapter
from x402.clients.base import x402Client

# Create client + session with adapter
client = x402Client(private_key, facilitator_url)
session = requests.Session()
session.mount('http://', x402HTTPAdapter(client))

# Normal requests API!
response = session.post(url, json={"prompt": "..."})
```

In [ ]:
# Schritt 2: Private Key Check
print("\n" + "=" * 60)
print("SCHRITT 2: Private Key Configuration")
print("=" * 60)

if private_key:
    print(f"\n✅ Private Key geladen aus .env")
    print(f"   Key: {private_key[:10]}...{private_key[-4:]}")
    print(f"\n💡 Dieser Key wird für x402 Payment Signing verwendet")
    print(f"   Netzwerk: {NETWORK_NAME}")
    print(f"   Chain ID: {CHAIN_ID}")
else:
    print("\n⚠️  PRIVATE KEY NICHT GEFUNDEN!")
    print("   Setze TEST_WALLET_PRIVATE_KEY in scw_js/.env")
    print("\n💡 Setup:")
    print("   1. Erstelle Wallet (z.B. MetaMask)")
    print("   2. Exportiere Private Key")
    print("   3. Hole Testnet USDC vom Faucet (für Sepolia)")
    print("   4. Setze TEST_WALLET_PRIVATE_KEY in scw_js/.env")

## Schritt 2: Private Key Configuration

Der Private Key wird automatisch aus `scw_js/.env` geladen (Variable: `TEST_WALLET_PRIVATE_KEY`).

**Netzwerk-Wahl:**
- **Testnet (Standard):** `USE_MAINNET = False`
  - Optimism Sepolia (Chain ID: 11155420)
  - Kein echtes Geld - zum Testen
  - USDC Faucet: [Optimism Sepolia Faucet](https://faucet.circle.com/)

- **Mainnet:** `USE_MAINNET = True`
  - Optimism Mainnet (Chain ID: 10)
  - ⚠️ **VORSICHT: Verwendet echtes USDC!**

**x402 Python Client:**
Der `x402_requests()` Client übernimmt automatisch:
- EIP-3009 Transfer Authorization Signing
- Netzwerk-Detection (basierend auf Service 402 Response)
- Payment Verification via Facilitator

In [17]:
# Schritt 1: Erste Anfrage OHNE Payment
print("=" * 60)
print("SCHRITT 1: Request ohne Payment (sollte 402 zurückgeben)")
print("=" * 60)

payload = {
    "prompt": PROMPT
}

try:
    response = requests.post(SERVICE_URL, json=payload, timeout=5)
    
    print(f"\n📬 Status Code: {response.status_code}")
    
    if response.status_code == 402:
        print("✅ Erwartete 402 Payment Required Response erhalten!\n")
        
        # Parse Response Body
        body = response.json()
        print("📄 Response Body:")
        pprint(body, width=80, indent=2)
        
        # Extract X-Payment header (x402 v2)
        x_payment_header = response.headers.get('X-Payment', response.headers.get('x-payment'))
        if x_payment_header:
            print("\n🔗 X-Payment Header (x402 v2 Payment Instructions):")
            payment_instructions = json.loads(x_payment_header)
            pprint(payment_instructions, width=80, indent=2)
            
            # Store for next step
            if payment_instructions.get('accepts'):
                accept = payment_instructions['accepts'][0]
                recipient = accept.get('payTo')
                amount = accept.get('amount')
                asset = accept.get('asset')
                network = accept.get('network')
                
                print(f"\n💡 Payment Details:")
                print(f"   - Network: {network}")
                print(f"   - Asset: {asset} (USDC)")
                print(f"   - Recipient: {recipient}")
                print(f"   - Amount: {amount} (in smallest unit)")
                print(f"\n📝 Next: Use x402 Python client to create payment")
        else:
            print("\n⚠️  Kein X-Payment Header gefunden!")
    else:
        print(f"❌ Unerwarteter Status: {response.status_code}")
        print(f"Response: {response.text[:200]}")
        
except requests.exceptions.ConnectionError:
    print("❌ Verbindungsfehler!")
    print("   Ist der Service gestartet? → cd scw_js && node genimg_x402_token.js")
except Exception as e:
    print(f"❌ Error: {e}")

SCHRITT 1: Request ohne Payment (sollte 402 zurückgeben)

📬 Status Code: 402
✅ Erwartete 402 Payment Required Response erhalten!

📄 Response Body:
{ 'error': 'Payment required',
  'message': 'Please provide USDC payment to generate your image',
  'payment': { 'accepts': [ { 'amount': '1000',
                              'asset': '0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85',
                              'extra': {'name': 'USDC', 'version': '2'},
                              'maxTimeoutSeconds': 60,
                              'network': 'eip155:10',
                              'payTo': '0xAAEBC1441323B8ad6Bdf6793A8428166b510239C',
                              'scheme': 'exact'},
                            { 'amount': '1000',
                              'asset': '0x5fd84259d66Cd46123540766Be93DFE6D43130D7',
                              'extra': {'name': 'USDC', 'version': '2'},
                              'maxTimeoutSeconds': 60,
                              'network': 'ei

In [21]:
import requests
import json
from pprint import pprint

# Test: Sind die Services erreichbar?
print("🔍 Service Verfügbarkeit Check\n")

# Check Facilitator (Production) - Use /supported endpoint
try:
    resp = requests.get(f"{FACILITATOR_URL}/supported", timeout=2)
    if resp.status_code == 200:
        supported = resp.json()
        print(f"✅ Facilitator: {FACILITATOR_URL} - Erreichbar (Production)")
        print(f"   Unterstützte Networks: {', '.join(supported.get('networks', []))}")
    else:
        print(f"⚠️  Facilitator: {FACILITATOR_URL} - Status {resp.status_code}")
except Exception as e:
    print(f"❌ Facilitator: {FACILITATOR_URL} - NICHT ERREICHBAR")
    print(f"   → Check Production Status ({e})")

# Check Service (Local)
try:
    resp = requests.post(SERVICE_URL, json={"prompt": "test"}, timeout=2)
    print(f"✅ Service: {SERVICE_URL} - Erreichbar (Status {resp.status_code})")
except:
    print(f"❌ Service: {SERVICE_URL} - NICHT ERREICHBAR")
    print("   → cd scw_js && NODE_ENV=test node genimg_x402_token.js")

🔍 Service Verfügbarkeit Check

✅ Facilitator: https://facilitator.fretchen.eu - Erreichbar (Production)
   Unterstützte Networks: 
✅ Service: http://localhost:8082 - Erreichbar (Status 402)


## Zusammenfassung

**Was passiert hinter den Kulissen:**

1. **Erster Request (ohne Payment):**
   - Service erstellt 402 mit x402 v2 Payment Header
   - Header enthält: USDC Address, Amount, Recipient, Network

2. **x402 Client Handling:**
   - Client extrahiert Payment Requirements aus X-Payment Header
   - Client erstellt EIP-3009 Transfer Authorization
   - Client signiert mit deinem Private Key
   - Client sendet Request mit `x-payment-authorization` Header

3. **Service Verification:**
   - Service sendet Payment an Facilitator `/verify`
   - Facilitator prüft:
     - EIP-3009 Signature gültig?
     - Payer hat genug USDC Balance?
     - Amount & Recipient korrekt?
   - Bei Success: Service generiert Bild

4. **NFT Minting:**
   - Service mintet NFT an Server Wallet
   - Service transferiert NFT an Client (Payment Payer)
   - Token ID aus Transaction Logs extrahiert

5. **Settlement (Async):**
   - Service sendet Payment an Facilitator `/settle`
   - Facilitator führt on-chain Transfer durch (später)
   - USDC wird vom Payer zum Server transferiert

**Vorteile:**
- ✅ Kein MetaMask/Wallet Connect nötig
- ✅ Automatisches Payment Signing
- ✅ Retry-Logic bei 402
- ✅ Server-side NFT Minting (garantiert)
- ✅ Async Settlement (schnelle Response)

## Troubleshooting

**"insufficient_funds" Error:**
- Check: Hat der Payer genug USDC Balance?
- Check: Ist das richtige Netzwerk konfiguriert? (Optimism Mainnet)
- Check: Unterstützt x402 das Netzwerk? (Optimism seit v1.0.1 supported)

**Facilitator Verification fehlschlägt:**
- Check: Ist Production Facilitator erreichbar? (https://facilitator.fretchen.eu)
- Check: Ist der Payer whitelisted? (Production hat Whitelist)
- Check: Stimmt die Chain ID? (Optimism = 10)

**Service Connection Error:**
- Check: Läuft der Service? (`cd scw_js && node genimg_x402_token.js`)
- Check: Ist NFT_WALLET_PRIVATE_KEY in .env gesetzt?
- Check: Ist BFL_API_TOKEN korrekt?
- Check: Ist FACILITATOR_URL in .env gesetzt?

**NFT Mint fehlschlägt:**
- Check: Hat Server Wallet genug ETH für Gas?
- Check: Ist NFT Contract auf Optimism deployed?
- Check: Ist RPC erreichbar?